In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from keras.applications import VGG19, VGG16
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from PIL import Image
import keras
from keras import Sequential, losses, optimizers, Input
from keras.layers import Dense, Conv2D, MaxPool2D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pywt
from scipy import signal


Using TensorFlow backend.
/Users/maahichatterjee/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/maahichatterjee/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/maahichatterjee/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/maahich

In [2]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=90):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "ButterworthKalmanGraphs"
    path1= "ButterworthKalmanGraphs/Cropped"
    data = [] 
    curated_data = {"label":[], "segment":[]}
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        if subject_name  ==".ipynb_checkpoints":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
        print(labelData)
        i=0
        for items in os.listdir(path+"/"+subject_name):
            if items == ".DS_Store":
                continue
            if items.endswith(".png"):
                #i=i+1
                #print(i)
                try:
                    im2 = cv2.imread(path+"/"+subject_name+"/"+items)
                    #plt.imshow(im2)
                    #plt.show()
                    crop_img = im2[30:20+235, 50:50+342]
                    im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    #print(type(crop_img))
                    #plt.imshow(crop_img)
                    #plt.show()
                    

              #curated_data['segments'].append(df1)
                    #curated_data['scalogram'].append(crop_img)
                    curated_data['segment'].append(im)
                    curated_data['label'].append(labelData)
              #print(df1)
                except:
                      df = None  
        #data.append(data_model) # Save all the data

    return curated_data


In [3]:
data = load_data(number_of_items=48)

Going through subject:person_123
person_123
Going through subject:person_124
person_124
Going through subject:person_112
person_112
Going through subject:person_115
person_115
Going through subject:person_114
person_114
Going through subject:person_113
person_113
Going through subject:person_122
person_122
Going through subject:person_200
person_200
Going through subject:person_207
person_207
Going through subject:person_231
person_231
Going through subject:person_209
person_209
Going through subject:person_208
person_208
Going through subject:person_230
person_230
Going through subject:person_201
person_201
Going through subject:person_223
person_223
Going through subject:person_212
person_212
Going through subject:person_215
person_215
Going through subject:person_214
person_214
Going through subject:person_213
person_213
Going through subject:person_222
person_222
Going through subject:person_100
person_100
Going through subject:person_107
person_107
Going through subject:person_109

In [4]:
temp=np.array(data['segment'])
print(temp.shape)

(50605, 224, 224)


In [5]:
encoder = LabelEncoder()
encoder.fit(data['label'])

LabelEncoder()

In [6]:
data['label'] = encoder.transform(data['label'])

In [7]:
all_the_classes = encoder.classes_
mapping = {0: 'person_100',
 1: 'person_101',
 2: 'person_102',
 3: 'person_103',
 4: 'person_104',
 5: 'person_106',
 6: 'person_107',
 7: 'person_108',
 8: 'person_109',
 9: 'person_111',
 10: 'person_112',
 11: 'person_113',
 12: 'person_114',
 13: 'person_115',
 14: 'person_116',
 15: 'person_117',
 16: 'person_118',
 17: 'person_119',
 18: 'person_121',
 19: 'person_122',
 20: 'person_123',
 21: 'person_124',
 22: 'person_200',
 23: 'person_202',
 24: 'person_203',
 25: 'person_205',
 26: 'person_207',
 27: 'person_208',
 28: 'person_209',
 29: 'person_210',
 30: 'person_212',
 31: 'person_213',
 32: 'person_214',
 33: 'person_215',
 34: 'person_217',
 35: 'person_219',
 36: 'person_220',
 37: 'person_221',
 38: 'person_222',
 39: 'person_223',
 40: 'person_228',
 41: 'person_230',
 42: 'person_231',
 43: 'person_232',
 44: 'person_233',
 45: 'person_234',
 46: 'person_105',
 47: 'person_201',
          }

#print(mapping)


# Get emotion from class number   
def get_name_from_class(class_number):
    """
        gets the corresponding subject from the class
    """
    if mapping.get(class_number,None):
        return mapping.get(class_number)
    else:
        return -1 # No such class

In [8]:
# Extracting features and labels
features = np.array(data['segment'])
labels = data['label']

In [9]:
print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

Shape of features:  (50605, 224, 224)
Shape of labels:  (50605,)


In [10]:
# Changing
labels = to_categorical(labels, num_classes=len(mapping))

In [11]:
# Building model
model = Sequential()
model.add(Conv2D(64, kernel_size=5, input_shape=(224, 224, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=5, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(150, activation="relu"))
model.add(Dense(120, activation="relu"))
model.add(Dense(100, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(80, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(len(mapping), activation="softmax"))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 220, 220, 64)      1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 64)      102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 53, 53, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 179776)           

In [13]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.RMSprop(),metrics=['accuracy'])

In [14]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, shuffle=True, random_state=17)

In [15]:
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.25, random_state=1)

In [16]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)

In [17]:
# Scaling
features_train = features_train.astype("float32")
features_test = features_test.astype("float32")

In [18]:
features_val = features_val.astype("float32")

In [19]:
features_train = features_train / 1/255
features_test = features_test / 1/255

In [20]:
features_val = features_val / 1/255

In [21]:
features_train = features_train.reshape(len(features_train), 224, 224, 1)
features_test = features_test.reshape(len(features_test), 224, 224, 1)

In [22]:
features_val = features_val.reshape(len(features_val), 224, 224, 1)

In [24]:
# Training
model.fit(features_train, labels_train, epochs=100, batch_size=16, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

Train on 28464 samples, validate on 12652 samples
Epoch 1/100
28464/28464 [==============================] - 4452s 156ms/step - loss: 1.3111 - accuracy: 0.6564 - val_loss: 0.5490 - val_accuracy: 0.8499

Epoch 2/100
28464/28464 [==============================] - 4362s 153ms/step - loss: 0.4929 - accuracy: 0.8694 - val_loss: 0.4122 - val_accuracy: 0.8831
Epoch 3/100
28464/28464 [==============================] - 4635s 163ms/step - loss: 0.3811 - accuracy: 0.8945 - val_loss: 0.3931 - val_accuracy: 0.8921
Epoch 4/100
28464/28464 [==============================] - 5189s 182ms/step - loss: 0.3400 - accuracy: 0.9068 - val_loss: 0.3930 - val_accuracy: 0.8964
Epoch 5/100
28464/28464 [==============================] - 4829s 170ms/step - loss: 0.3120 - accuracy: 0.9133 - val_loss: 0.3590 - val_accuracy: 0.9045
Epoch 6/100
28464/28464 [==============================] - 4333s 152ms/step - loss: 0.2917 - accuracy: 0.9176 - val_loss: 0.3974 - val_accuracy: 0.9101
Epoch 7/100
28464/28464 [============

In [25]:
model.save("2DCNNButterworthKalmanGraphs.h5") # Saving the model

In [26]:
# Test an image
def test_image(image,label):
    #print(image.shape)
    prediction = np.argmax(model.predict(image.reshape(1,224,224,1)))
    result_predict=get_emotion_from_class(prediction)
    actual=np.argmax(label)
    result_actual=get_emotion_from_class(actual)
    #print(prediction)
    #print(actual)
    return result_actual,result_predict

    
# Get emotion from class number   
def get_emotion_from_class(class_number):
    """
        gets the corresponding label from the class
    """
    if mapping.get(class_number,None):
      return mapping.get(class_number)
    else:
      return -1 # No such class

In [28]:
countFP=0
for i in range(len(features_val)):
    actual,predicted=test_image(features_val[i],labels_val[i])
    print("Model predicts: "+predicted+" and actual label is: "+actual)
    if(predicted!=actual):
        print("Wrong prediction")
        countFP=countFP+1
    print("-------------------------------------------------------------")
  
print("Total False Positives: "+str(countFP))
print("Total Positives: "+str(len(features_val)))
percent=(countFP/len(features))*100
print("FP %: "+str(percent))

Model predicts: person_232 and actual label is: person_232
-------------------------------------------------------------
Model predicts: person_202 and actual label is: person_202
-------------------------------------------------------------
Model predicts: person_124 and actual label is: person_124
-------------------------------------------------------------
Model predicts: person_201 and actual label is: person_210
Wrong prediction
-------------------------------------------------------------
Model predicts: person_209 and actual label is: person_209
-------------------------------------------------------------
Model predicts: person_121 and actual label is: person_115
Wrong prediction
-------------------------------------------------------------
Model predicts: person_214 and actual label is: person_214
-------------------------------------------------------------
Model predicts: person_230 and actual label is: person_230
------------------------------------------------------------

Model predicts: person_210 and actual label is: person_210
-------------------------------------------------------------
Model predicts: person_234 and actual label is: person_234
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts: person_113 and actual label is: person_113
-------------------------------------------------------------
Model predicts: person_112 and actual label is: person_112
-------------------------------------------------------------
Model predicts: person_123 and actual label is: person_123
-------------------------------------------------------------
Model predicts: person_212 and actual label is: person_212
-------------------------------------------------------------
Model predicts: person_124 and actual label is: person_124
-------------------------------------------------------------
Model predicts: person_230 and a

Model predicts: person_108 and actual label is: person_108
-------------------------------------------------------------
Model predicts: person_108 and actual label is: person_108
-------------------------------------------------------------
Model predicts: person_100 and actual label is: person_100
-------------------------------------------------------------
Model predicts: person_214 and actual label is: person_214
-------------------------------------------------------------
Model predicts: person_106 and actual label is: person_106
-------------------------------------------------------------
Model predicts: person_201 and actual label is: person_201
-------------------------------------------------------------
Model predicts: person_109 and actual label is: person_109
-------------------------------------------------------------
Model predicts: person_100 and actual label is: person_100
-------------------------------------------------------------
Model predicts: person_228 and a

Model predicts: person_108 and actual label is: person_108
-------------------------------------------------------------
Model predicts: person_213 and actual label is: person_213
-------------------------------------------------------------
Model predicts: person_118 and actual label is: person_118
-------------------------------------------------------------
Model predicts: person_219 and actual label is: person_219
-------------------------------------------------------------
Model predicts: person_222 and actual label is: person_222
-------------------------------------------------------------
Model predicts: person_119 and actual label is: person_119
-------------------------------------------------------------
Model predicts: person_220 and actual label is: person_220
-------------------------------------------------------------
Model predicts: person_203 and actual label is: person_203
-------------------------------------------------------------
Model predicts: person_214 and a

Model predicts: person_214 and actual label is: person_214
-------------------------------------------------------------
Model predicts: person_232 and actual label is: person_232
-------------------------------------------------------------
Model predicts: person_103 and actual label is: person_103
-------------------------------------------------------------
Model predicts: person_118 and actual label is: person_118
-------------------------------------------------------------
Model predicts: person_234 and actual label is: person_234
-------------------------------------------------------------
Model predicts: person_212 and actual label is: person_212
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts: person_230 and actual label is: person_230
-------------------------------------------------------------
Model predicts: person_207 and a

...

Model predicts: person_232 and actual label is: person_232
-------------------------------------------------------------
Model predicts: person_114 and actual label is: person_114
-------------------------------------------------------------
Model predicts: person_202 and actual label is: person_202
-------------------------------------------------------------
Model predicts: person_210 and actual label is: person_210
-------------------------------------------------------------
Model predicts: person_230 and actual label is: person_230
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts: person_214 and actual label is: person_214
-------------------------------------------------------------
Model predicts: person_116 and actual label is: person_116
-------------------------------------------------------------
Model predicts: person_213 and a

Model predicts: person_217 and actual label is: person_217
-------------------------------------------------------------
Model predicts: person_230 and actual label is: person_230
-------------------------------------------------------------
Model predicts: person_219 and actual label is: person_219
-------------------------------------------------------------
Model predicts: person_113 and actual label is: person_113
-------------------------------------------------------------
Model predicts: person_124 and actual label is: person_124
-------------------------------------------------------------
Model predicts: person_107 and actual label is: person_107
-------------------------------------------------------------
Model predicts: person_214 and actual label is: person_214
-------------------------------------------------------------
Model predicts: person_121 and actual label is: person_121
-------------------------------------------------------------
Model predicts: person_201 and a

Model predicts: person_217 and actual label is: person_217
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts: person_112 and actual label is: person_112
-------------------------------------------------------------
Model predicts: person_114 and actual label is: person_114
-------------------------------------------------------------
Model predicts: person_202 and actual label is: person_202
-------------------------------------------------------------
Model predicts: person_230 and actual label is: person_104
Wrong prediction
-------------------------------------------------------------
Model predicts: person_234 and actual label is: person_234
-------------------------------------------------------------
Model predicts: person_116 and actual label is: person_116
-------------------------------------------------------------
Model predicts:

Model predicts: person_228 and actual label is: person_228
-------------------------------------------------------------
Model predicts: person_100 and actual label is: person_100
-------------------------------------------------------------
Model predicts: person_202 and actual label is: person_202
-------------------------------------------------------------
Model predicts: person_101 and actual label is: person_101
-------------------------------------------------------------
Model predicts: person_213 and actual label is: person_213
-------------------------------------------------------------
Model predicts: person_121 and actual label is: person_103
Wrong prediction
-------------------------------------------------------------
Model predicts: person_123 and actual label is: person_123
-------------------------------------------------------------
Model predicts: person_111 and actual label is: person_111
-------------------------------------------------------------
Model predicts:

Model predicts: person_122 and actual label is: person_122
-------------------------------------------------------------
Model predicts: person_104 and actual label is: person_220
Wrong prediction
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts: person_121 and actual label is: person_121
-------------------------------------------------------------
Model predicts: person_200 and actual label is: person_200
-------------------------------------------------------------
Model predicts: person_203 and actual label is: person_203
-------------------------------------------------------------
Model predicts: person_102 and actual label is: person_102
-------------------------------------------------------------
Model predicts: person_215 and actual label is: person_215
-------------------------------------------------------------
Model predicts:

In [29]:
score = model.evaluate(features_val, labels_val, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))